# Поиск похожих изображений

Импорт необходимых библиотек

In [26]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from skimage import *
import pandas as pd
import glob
import joblib
import base64

Cоздаем базу дескрипторов для обучения и сохраним все в один вектор

In [2]:
arr=[]
k=0
for i in glob.glob('images/train/*.jpg'):
    img = cv.imread(i)
    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create()
    kp, des = sift.detectAndCompute(gray,None)
    for j in range(des.shape[0]):
        arr.append(des[j])

        
deses=[]
deses = np.array(arr)
deses.shape

Кластеризация изображений по одному общему вектору

In [4]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=2048)
model.fit(deses)

KMeans(n_clusters=2048)

Сохраним модель для использования уже в рабочей части программы

In [5]:
joblib.dump(model, 'model/model.sav')

['model/model.sav']

Загрузка расчитанной модели, чтобы не ждать, пока рассчитается новая модель 

In [6]:
loaded_model = joblib.load('model/model.sav')
loaded_model.predict(des)

array([1556, 1569, 1640,  125,  469, 1605, 1808,  568,  137,  879, 1682,
       1526, 2000,   51, 1605,  232,   51,  842,  402, 1313,  390, 1868,
         23,  386,  787,  891, 1526, 1350, 1427,  632,  276,   15,  489,
       1835,  315,  934,  231,  842,  842, 1486, 1530, 1832, 1951,  547,
       2003,  905,  905, 1434,  147, 1410, 1309,  648, 1336,  493, 1628,
       1966,  394,  808, 1739, 1413, 1870, 1401, 1413,  405, 1842, 1486,
       1634,  810, 1307,  540,  414,  779, 1117,  193, 1222,  366,  918,
       1567, 2027,  124, 2044,  695, 1859, 1806, 1859, 1353,  836,  947,
        634, 1636,  486, 2019, 1023, 1762,  348,  590, 1806,  891, 1827,
       1965, 1684,  257, 1532,  890, 1375, 1680,  976, 1667,  894, 1754,
       1946,  929,  450,  431, 1694, 1300,  198, 1595,  774, 1043,  860,
         82,  621,  460,  378, 1675, 1940,  307, 1687, 1770,  546,  465,
       1700, 1578, 1072,  510,  518,  290, 1646,  870, 1747, 1525, 1033,
       1823, 1714, 1825, 1914,  884, 1199,  704,  1

Вывод имени изображения и принадлежность каждого бина к определенному кластеру по обученной модели и сохраним все в таблицу

In [7]:
img_name = []
embeddings = []
k=0
for i in glob.glob('images/voc/*.jpg'):
    img_name.append(i.split("/")[-1][4:20])
    img = cv.imread(i)
    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create()
    kp, des = sift.detectAndCompute(gray,None)
    embeddings.append(base64.b64encode(loaded_model.predict(des)))
    
images_info = {'link': img_name,'embedding': embeddings}
images_df = pd.DataFrame(data=images_info)
images_df.to_csv('output.csv',index=False) 

Чтение полученного файла для проверки правильности вывода. Перевод данных в десятичный формат.

*Было сохранеие в виде байта, а не строки из-за особенности сохранения большой строки в таблицу*

In [22]:
data0 = pd.read_csv("output.csv", delimiter=',')
data0['embedding'] = data0['embedding'].apply(lambda x: np.frombuffer(base64.b64decode(bytes(x[2:-1],encoding='ascii')), dtype=np.int32))
data0

,link,embedding
0,2007_000027.jpg,"[1459, 135, 1849, 288, 1625, 1789, 1989, 646, ..."
1,2007_000032.jpg,"[517, 1234, 1888, 1091, 1978, 1830, 1888, 1017..."
2,2007_000033.jpg,"[942, 371, 411, 632, 478, 266, 75, 23, 499, 13..."
3,2007_000039.jpg,"[521, 1526, 917, 674, 674, 674, 1227, 1033, 11..."
4,2007_000042.jpg,"[1833, 640, 651, 1393, 208, 239, 1841, 2020, 1..."
...,...,...
7436,2012_004273.jpg,"[776, 303, 149, 149, 255, 1303, 577, 577, 308,..."
7437,2012_004292.jpg,"[897, 652, 449, 834, 44, 44, 1585, 1418, 1585,..."
7438,2012_004293.jpg,"[343, 1891, 2006, 379, 1165, 135, 1279, 237, 1..."
7439,2012_004310.jpg,"[417, 210, 249, 992, 1057, 742, 1330, 1855, 13..."


Расчет количества вхождения бинов в определенный вид кластера и перевод обратно в байтовый формат, после чего сохраненяем в таблицу

In [23]:
matrix=np.zeros((len(data0),2048),dtype=np.uint8)
for j in range (len(data0)):
    for i in range (len(data0['embedding'][j])):
        matrix[j,data0['embedding'][j][i]]+=1
    data0['embedding'][j]=(base64.b64encode(matrix[j]))
data0.to_csv('output1.csv',index=False) 
data0['embedding']

0       b'AQMBAQAAAQIBAAACAAEBAAABAQQAAgIDAgAAAAABAQMC...
1       b'AAAAAAEAAAABAgAAAAAAAAAAAAAAAAAAAQABAAABAAAA...
2       b'AAAAAAAAAAABAwEAAAAAAQAAAAAAAAEBAAAAAgAAAAAA...
3       b'AAAAAAAABAAAAQAAAAQAAAAAAAAAAAAAAAAAAAABAAAA...
4       b'AQAAAQAAAAEAAQAAAAEAAQAAAQABAAEBAAAAAAEAAAAA...
                              ...                        
7436    b'AAEAAAAAAAACAwEAAAAAAAABAAAFAAEAAAAAAQAAAQAA...
7437    b'AAEAAAAAAAACAAECAAEAAAABAgAGAAAAAAEBAQEAAQEA...
7438    b'AAAAAAAAAAAAAAEAAAABAQAAAAEAAgACAAAAAAAAAAAA...
7439    b'AAEAAAAAAAABAAABAAAAAgABAQADAQAAAgEAAAAAAQAB...
7440    b'AQABAAIAAAAAAQIAAAAAAQEBAQAEAAAAAAMCAAAAAAAA...
Name: embedding, Length: 7441, dtype: object